# Week 2 - Gurobi Introduction 🚀

# 0 - Learning Outcomes
 - Intro, setup and basics.
 - Introduction to numpy and pandas.
 - Plotting.

# 1 - What is Gurobi?
State-of-the-art solvers for mathematical programming problems:
- Linear programming (LP).
- Mixed-integer linear programming (MILP).
- Mixed-integer quadratic programming (MIQP).
- Quadratic programming (QP).
- Quadratically constrained programming (QCP).
- Mixed-integer quadratically constrained programming (MIQCP).

# 2 - Setup
You can find the full documentation for installing on the website: https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python

We can access Gurobi using it's Python API. You can use pip or conda to install Gurobi into your currently active Python environment.

```python -m pip install gurobipy```

Upon sucessfull installation, you should see `gurobipy` amongst the installed packages listed by typing:

```python -m pip list```

In order to use Gurobi, you need to get a free academic license: https://support.gurobi.com/hc/en-us/articles/360040541251-How-do-I-obtain-a-free-academic-license

Once you have downloaded the licence, you can find out where to place the `gurobi.lic` file here: https://support.gurobi.com/hc/en-us/articles/360013417211-Where-do-I-place-the-Gurobi-license-file-gurobi-lic-

# 3 - Gurobi Basics
Suppose we want to solve the following:

$$
  \begin{align}
      \textrm{maximize} \quad &x + y \\
      \textrm{subject to} \quad &2x - 2y \leq 1 \\
      &-8x + 10y \leq 13 \\
      &x, y \in \mathbb{Z} \\
  \end{align}
$$

## 3.1 - Importing the module
- It is common to make Gurobi methods available through a `gp.` prefix.
- We also make the `GRB` class available without a prefix as we will use this alot to set attributes and parameters.

In [1]:
import gurobipy as gp
from gurobipy import GRB

## 3.2 - Gurobi environments
- Building and solving a Gurobi model typically requires an environment.
- There is also the default environment which simplifies experimentation, but it is best practice to create an environment explicitly.

```
with gp.Env() as env, gp.Model(env=env) as model:
    x = model.addVar()
    ...
    model.optimize()
    ...
```


- At the end of the block, the environment is closed.
- The resources are then returned to the compute.

## 3.3 - Building a model
A Gurobi model holds a single optimization problem, comprising:
- Variables
- Constraints
- Bounds
- Constants

In [22]:
model = gp.Model("my_mip_model") 

## 3.4 - Adding variables
- We add a variable to the model using the `Model.addVar` method.
- You can add more than one variable at a time with `Model.addVars`.
- A variable needs to be associated with a model.
- Each variable is asssigned a type and a name. 

In [23]:
x = model.addVar(vtype=GRB.INTEGER, name="x")
y = model.addVar(vtype=GRB.INTEGER, name="y")

## 3.5 - Adding constraints
- We add a constraint to the model using the `Model.addConstr` method.
- We can also name constraints.

In [ ]:
model.addConstr(2 * x - 2 * y <= -1, "c0")
model.addConstr(-8 * x + 10 * y <= 13, "c1")

## 3.6 - Objective
- We add the objective to the model using the `Model.setObjective` method.
- Here we can indicate whether it is a minimization or maximization problem.

In [25]:
model.setObjective(x + y, GRB.MAXIMIZE)

## 3.7 - Write optimization model
- Using the `Model.write` method we can create an LP file to view our model. 
- We can open the file and check if there are errors in our model formulation - very helpful for debugging

In [ ]:
model.write("model.lp")

## 3.8 - Solving the model

- Now that we have built our model, we can optimize it using the `Model.optimize` method.

In [ ]:
model.optimize()

## 3.8 - Reporting results

- Once the model has solved (hopefully sucessfully), we can access the results.

In [ ]:
for v in model.getVars():
    print('%s %g' % (v.VarName, v.X))

- We can also query the ObjVal attribute on the model to obtain the objective value for the current solution

In [ ]:
print('Obj: %g' % model.ObjVal)

# 4 - Matrix model
We can re-write our problem in canonical form as:

$$
  \begin{align}
      \textrm{maximize} \quad &c^{\top} x \\
      \textrm{subject to} \quad &Ax \leq b \\
      &x \in \mathbb{Z}^3 \\
  \end{align}
$$

In [41]:
import numpy as np
import scipy.sparse as sp

model = gp.Model("my_mip_model") 

- A matrix variable is added through the Model.addMVar method on a model object. In this case the matrix variable consists of a 1-D array of 3 binary variables. Variables are always associated with a particular model.

In [42]:
x = model.addMVar(shape=2, vtype=GRB.INTEGER, name="x")

- The next step in the example is to add our two linear constraints.
- This is done by building a sparse matrix that captures the constraint matrix:

In [43]:
# Build (sparse) constraint matrix
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

In [44]:
A = np.array([[2, -2], [-8, 10]])

The matrix has two rows, one for each constraint, and three columns, one for each variable in our matrix variable. The row and col arrays gives the row and column indices for the 5 non-zero values in the sparse matrix, respectively. The val array gives the numerical values. Note that we multiply the greater-than constraint by -1 to transform it to a less-than constraint.

We also capture the right-hand side in a NumPy array:

In [45]:
# Build rhs vector
rhs = np.array([-1, 13])

We then use the overloaded @ operator to build a linear matrix expression, and then use the overloaded less-than-or-equal operator to add two constraints (one for each row in the matrix expression) using Model.addConstr:

In [ ]:
model.addConstr(A @ x <= rhs, name="c")

The next step is to set the optimization objective:
The objective is built here by computing a dot product between a constant vector and our matrix variable using the overloaded @ operator. Note that the constant vector must have the same length as our matrix variable.

The second argument indicates that the sense is maximization.


In [47]:
c = np.array([[1], [1]])
model.setObjective(c.T @ x, GRB.MAXIMIZE)

After the model setup we can check the formulation to see if we made an error. It shows all single constraints:

In [48]:
model.write("model.lp")

Now that the model has been built, the next step is to optimize it:



In [ ]:
model.optimize()

Once the optimization is complete, we can query the values of the attributes. In particular, we can query the X variable attributes to obtain the solution value for each variable:

In [ ]:
print(x.X)

We can also query the ObjVal attribute on the model to obtain the objective value for the current solution:

In [ ]:
print('Obj: %g' % model.ObjVal)